<a href="https://colab.research.google.com/github/DanishGada/GANS/blob/main/MNIST_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
import matplotlib
from torchvision.utils import make_grid, save_image
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import cv2
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook
from tqdm import tqdm
import glob
import math
import itertools
import imageio
import natsort

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
epochs = 1000
sample_size = 64 # fixed sample size
batch_size = 128
# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 28
# Number of channels in the training images. For color images this is 3
nc = 1
# Size of z latent vector (i.e. size of generator input)
z_dim = 100
# Size of feature maps in generator
ngf = 32
# Size of feature maps in discriminator
ndf = 32
# Number of training epochs
num_epochs = 5
# Learning rate for optimizers
lr = 0.0002
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,)),
])
to_pil_image = transforms.ToPILImage()

In [ ]:
train_data = datasets.MNIST(
    root='../input/data',
    train=True,
    download=True,
    transform=transform
)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../input/data/MNIST/raw/train-images-idx3-ubyte.gz to ../input/data/MNIST/raw




Extracting ../input/data/MNIST/raw/train-labels-idx1-ubyte.gz to ../input/data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../input/data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../input/data/MNIST/raw




Extracting ../input/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../input/data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Since Batch Norm is Applied After Every Layer , Baises are set to Zero

In [ ]:
class Generator(nn.Module):
  def __init__(self,z_dim):
    super(Generator,self).__init__()

    self.main = nn.Sequential(
     nn.ConvTranspose2d(in_channels = z_dim, out_channels= ngf*4, kernel_size= 4, stride= 1, padding= 0, bias=False,),
     nn.BatchNorm2d(ngf * 4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
     nn.ReLU(True),

     nn.ConvTranspose2d(in_channels= ngf*4,out_channels= ngf*2 , kernel_size= 3 ,stride= 2, padding= 1 ,bias= False),
     nn.BatchNorm2d(ngf* 2 , eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
     nn.ReLU(True),

     nn.ConvTranspose2d(in_channels= ngf*2,out_channels= ngf , kernel_size= 4 ,stride= 2, padding= 1 ,bias= False),
     nn.BatchNorm2d(ngf, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
     nn.ReLU(True),

     nn.ConvTranspose2d(in_channels= ngf,out_channels= nc , kernel_size= 4 ,stride= 2, padding= 1 ,bias= False),
     nn.Tanh()
    )
  def forward(self, input):
    return self.main(input)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(in_channels = nc, out_channels= ndf, kernel_size= 4, stride= 2, padding= 1, bias=False,),
            nn.LeakyReLU(negative_slope= 0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(in_channels = ndf, out_channels= ndf*2, kernel_size= 4, stride= 2, padding= 1, bias=False,),
            nn.BatchNorm2d(ndf * 2,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.LeakyReLU(negative_slope= 0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(in_channels = ndf*2, out_channels= ndf*4, kernel_size= 3, stride= 2, padding= 1, bias=False,),
            nn.BatchNorm2d(ndf * 4,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.LeakyReLU(negative_slope= 0.2, inplace=True),

            nn.Conv2d(in_channels = ndf*4, out_channels= 1, kernel_size= 4, stride= 1, padding= 0, bias=False,),
            nn.Sigmoid()
        )
    def forward(self, input):
      return self.main(input).view(-1,1).squeeze(1)

In [ ]:


generator = Generator(z_dim).to(device)
discriminator = Discriminator().to(device)
print('##### GENERATOR #####')
print(generator)
print('######################')
print('\n##### DISCRIMINATOR #####')
print(discriminator)
print('######################')

##### GENERATOR #####
Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(32, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)
######################

##### DISCRIMINATOR #####
Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
generator.apply(weights_init)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(32, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)

In [ ]:
discriminator.apply(weights_init)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)

In [ ]:
optim_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optim_d = optim.Adam(discriminator.parameters(),lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()
losses_g = [] 
losses_d = [] 
images = [] 

In [ ]:
def label_real(size):
    data = torch.ones(size, 1).to(device)
    return data
    
def label_fake(size):
    data = torch.zeros(size, 1).to(device)
    return data

def save_generator_image(image, path):
    save_image(image, path)

def create_noise(sample_size, nz):
    return torch.randn(sample_size, nz,1,1).to(device)

In [ ]:
def train_discriminator(optimizer, data_real, data_fake):
    b_size = data_real.size(0)
    real_label = label_real(b_size)
    fake_label = label_fake(b_size)
    optimizer.zero_grad()
    # sEND REAL data
    output_real = discriminator(data_real)
    output_real =torch.reshape(output_real, (output_real.shape[0], 1))
    loss_real = criterion(output_real, real_label)
    #Send fake data
    output_fake = discriminator(data_fake)
    output_fake =torch.reshape(output_fake, (output_fake.shape[0], 1))
    loss_fake = criterion(output_fake, fake_label)
    #backprop
    loss_real.backward()
    loss_fake.backward()
    optimizer.step()
    return loss_real + loss_fake

In [ ]:
def train_generator(optimizer, data_fake):
    b_size = data_fake.size(0)
    real_label = label_real(b_size)
    optimizer.zero_grad() 
    output = discriminator(data_fake)
    # print(data_fake.shape,real_label.shape,output.shape)  
    output = torch.reshape(output , (output.shape[0],1))
    loss = criterion(output, real_label)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:

discriminator.train()

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)

In [ ]:
noise = create_noise(sample_size, z_dim).to(device)

In [ ]:
path= '/content/drive/MyDrive/Colab Notebooks/Cnn_output'

In [ ]:
epoch_loss_g=0
epoch_loss_d=0
losses_g = [] 
losses_d = [] 
images = [] 
k=2

'''
        In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch 
        accumulates the gradients on subsequent backward passes. This is convenient while training RNNs. So, 
        the default action is to accumulate (i.e. sum) the gradients on every loss.backward() call.
        
        Because of this, when you start your training loop, ideally you should zero out the gradients so that 
        you do the parameter update correctly. Else the gradient would point in some other direction than the 
        intended direction towards the minimum (or maximum, in case of maximization objectives).
        
        '''
for epoch in range(epochs):
    loss_g = 0.0
    loss_d = 0.0
    for bi, data in tqdm_notebook(enumerate(train_loader),total=int(len(train_data) / train_loader.batch_size)):
        image, _ = data
        b_size = len(image)
        image = image.to(device)
        # print(b_size)
        for step in range(k):
            data_fake = generator(create_noise(b_size,z_dim))
            data_real = image
            # print(data_fake.shape)

        loss_d += train_discriminator(optim_d, data_real, data_fake)
        data_fake = generator(create_noise(b_size, z_dim))
        loss_g += train_generator(optim_g, data_fake)
        # print('.',end="")
    generated_img = generator(noise)
    generated_img = make_grid(generated_img)
    save_generator_image(generated_img, f"/content/drive/MyDrive/Colab Notebooks/Cnn_output/gen_img{epoch}.png")
    images.append(generated_img)
    epoch_loss_g = loss_g / bi 
    epoch_loss_d = loss_d / bi
    losses_g.append(epoch_loss_g)
    losses_d.append(epoch_loss_d)
    
    print(f"Epoch {epoch + 1} of {epochs}")
    print(f"Generator loss: {epoch_loss_g:}, Discriminator loss: {epoch_loss_d:}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


KeyboardInterrupt: ignored

In [ ]:

def generate_images(epoch, path, fixed_noise, num_test_samples, netG, device, use_fixed=False):
    z = torch.randn(num_test_samples, 100, 1, 1, device=device)
    size_figure_grid = int(math.sqrt(num_test_samples))
    title = None
  
    if use_fixed:
        generated_fake_images = netG(fixed_noise)
        path += 'fixed_noise/'
        title = 'Fixed Noise'
    else:
        generated_fake_images = netG(z)
        path += 'variable_noise/'
        title = 'Variable Noise'
  
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(6,6))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i,j].get_xaxis().set_visible(False)
        ax[i,j].get_yaxis().set_visible(False)
    for k in range(num_test_samples):
        i = k//4
        j = k%4
        ax[i,j].cla()
        ax[i,j].imshow(generated_fake_images[k].data.cpu().numpy().reshape(28,28), cmap='Greys')
    label = 'Epoch_{}'.format(epoch+1)
    fig.text(0.5, 0.04, label, ha='center')
    fig.suptitle(title)
    fig.savefig(path+label+'.png')

def save_gif(path, fps):
    print(path + '*.png')
    images = glob.glob(path + '*.png')
    images = natsort.natsorted(images)
    gif = []

    for image in images:
        gif.append(imageio.imread(image))
    imageio.mimsave(path+'animated.gif', gif, fps=fps)

In [ ]:
 save_gif(path+'/' , 10)

/content/drive/MyDrive/Colab Notebooks/Cnn_output/*.png


In [ ]:
print("Model's state_dict:")
for param_tensor in generator.state_dict():
    print(param_tensor, "\t", generator.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optim_g.state_dict():
    print(var_name, "\t", optim_g.state_dict()[var_name])
print("Optimizer's state_dict:")
for var_name in optim_d.state_dict():
    print(var_name, "\t", optim_d.state_dict()[var_name])

Model's state_dict:
main.0.weight 	 torch.Size([100, 128, 4, 4])
main.1.weight 	 torch.Size([128])
main.1.bias 	 torch.Size([128])
main.1.running_mean 	 torch.Size([128])
main.1.running_var 	 torch.Size([128])
main.1.num_batches_tracked 	 torch.Size([])
main.3.weight 	 torch.Size([128, 64, 3, 3])
main.4.weight 	 torch.Size([64])
main.4.bias 	 torch.Size([64])
main.4.running_mean 	 torch.Size([64])
main.4.running_var 	 torch.Size([64])
main.4.num_batches_tracked 	 torch.Size([])
main.6.weight 	 torch.Size([64, 32, 4, 4])
main.7.weight 	 torch.Size([32])
main.7.bias 	 torch.Size([32])
main.7.running_mean 	 torch.Size([32])
main.7.running_var 	 torch.Size([32])
main.7.num_batches_tracked 	 torch.Size([])
main.9.weight 	 torch.Size([32, 1, 4, 4])
Optimizer's state_dict:
state 	 {0: {'step': 7, 'exp_avg': tensor([[[[-4.8585e-03,  2.2081e-03, -5.8133e-03,  1.7221e-03],
          [ 5.1960e-03,  7.0620e-04,  2.5706e-03, -5.5080e-04],
          [ 4.3584e-03, -4.0245e-04,  4.2010e-03, -3.2012e-0

In [ ]:
torch.save(generator.state_dict(), path + '/generator.pt' )
torch.save(optim_g.state_dict(), path + '/optim_g.pt')
torch.save(optim_d.state_dict(), path + '/optim_d.pt')
torch.save(discriminator.state_dict(), path + '/discriminator.pt')